In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests

In [22]:
# env 파일 로드
load_dotenv()

# openai, kakao rest api 키 로드
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
KAKAO_REST_API_KEY = os.getenv('KAKAO_RESTAPI_KEY')

In [15]:
def openai_message():
    client = OpenAI(api_key = openai_api_key)
    completion = client.chat.completions.create(
    model="gpt-4o-mini-search-preview",
    web_search_options={
        "user_location": {
            "type": "approximate",
            "approximate": {
                "country": "KR",  # 대한민국
                "city": "Chuncheon",  # 춘천
                "region": "Gangwon"  # 강원도
            }
        },
        "search_context_size": "medium"  # 검색 컨텍스트 크기 (low, medium, high 중 선택)
    },
    messages=[{
        "role": "user",
        "content": """지금부터 당신은 장소 리뷰어가 된다. 
아래 스팟 리스트에 대해서 google에 해당 스팟을 검색해 접근 가능한 블로그글, 리뷰글에 대해, 부가적인 내용없이 각 스팟에 대해 출력 형식에 맞춰 답변한다.

스팟 리스트
1. 리네 춘천
2. 브릭스피자 춘천
3. 오믈하우스 춘천
4. 송탄햄버거 춘천
5. 만석식당 춘천

출력 형식
장소 이름/긍정평가:n%/부정평가:n%
출처: {url}"""
    }]
)

    print(completion.choices[0].message.content)

In [9]:
def kakao_rest_api(longitude, latitude):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {'x': longitude, 'y': latitude}
    
    response = requests.get(url, headers=headers, params=params)
    
    # 데이터를 성공적으로 불러온 경우, code = 200
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [23]:
def search_by_category(longitude, latitude, category_code):
    url = 'https://dapi.kakao.com/v2/local/search/category.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {
        'category_group_code': category_code,
        'x': longitude,
        'y': latitude,
        'radius': 2000,  # 검색 반경 설정 (단위: 미터)
        'size': 15       # 한 페이지에 보여질 결과 개수
    }

    # API 요청
    response = requests.get(url, headers=headers, params=params)

    # 응답 처리
    if response.status_code == 200:
        return response.json()  # 성공적으로 데이터를 반환
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None  # 실패 시 None 반환

In [25]:
# 강원대학교 후문의 경도와 위도
longitude = "127.73983"  # 경도
latitude = "37.87760"    # 위도
category_code = "CE7"    # 카테고리 코드 (예: 카페)

result = search_by_category(longitude, latitude, category_code)
if result:
    print(result)
else:
    print("API 요청에 실패했습니다.")

{'documents': [{'address_name': '강원특별자치도 춘천시 교동 148-4', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페', 'distance': '595', 'id': '1815650178', 'phone': '010-8823-0403', 'place_name': '그라시아커피로스터스', 'place_url': 'http://place.map.kakao.com/1815650178', 'road_address_name': '강원특별자치도 춘천시 교동길9번길 10', 'x': '127.735366831268', 'y': '37.8816299468414'}, {'address_name': '강원특별자치도 춘천시 효자동 619-1', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페', 'distance': '354', 'id': '27408470', 'phone': '', 'place_name': '포지티브즈', 'place_url': 'http://place.map.kakao.com/27408470', 'road_address_name': '강원특별자치도 춘천시 서부대성로205번길 10', 'x': '127.741850488403', 'y': '37.8748322964497'}, {'address_name': '강원특별자치도 춘천시 효자동 601-11', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 테마카페 > 갤러리카페', 'distance': '389', 'id': '1565469687', 'phone': '', 'place_name': '예담카페', 'place_url': 'http://place.map.kakao